# Create winterYear tiled snowDynamics products

## Setup

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import xarray as xr
import os
import gc

import geopandas as gpd # To get HBL tile IDs

import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import PreProcess_Utils as pputil

sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/Modelling/Utilities')
import Snow_Utils as snow_util

In [2]:
from dask.distributed import LocalCluster

cluster = LocalCluster(n_workers = 10, threads_per_worker = 10, memory_limit = 0) # Refined through testing, removing memory_limit is important
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58863,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:58996,Total threads: 10
Dashboard: http://127.0.0.1:59014/status,Memory: 0 B
Nanny: tcp://127.0.0.1:58866,


2025-05-03 16:09:40,581 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 4f8165e3bf920587fb72dec816ddbbb7 initialized by task ('rechunk-merge-rechunk-transfer-3235283bbe9d03cd098717d1f350e758', 0, 0, 0, 25, 0, 0) executed on worker tcp://127.0.0.1:58995
2025-05-03 16:09:53,858 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 4f8165e3bf920587fb72dec816ddbbb7 deactivated due to stimulus 'task-finished-1746302993.5743015'
2025-05-03 16:12:23,398 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 4f8165e3bf920587fb72dec816ddbbb7 initialized by task ('rechunk-merge-rechunk-transfer-3235283bbe9d03cd098717d1f350e758', 0, 0, 0, 25, 0, 0) executed on worker tcp://127.0.0.1:59017
2025-05-03 16:12:33,819 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 4f8165e3bf920587fb72dec816ddbbb7 deactivated due to stimulus 'task-finished-1746303153.7796626'
2025-05-03 16:14:34,730 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 4f8165e3bf920587fb72dec816

In [3]:
tile_stats = pd.read_csv(r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\High Resolution Snow Dynamics\IMS (1 km)\Outputs\tiles_IMS_SnowDynamics4HLS.csv", index_col = 0)
tile_stats

,smin_avg,smax_avg,sstart_avg,send_avg,send_last,sstart_first
0,209.85715,34.500000,4.166666,61.500000,90.0,335.0
1,217.14285,37.166668,3.333333,68.500000,90.0,335.0
2,209.71428,34.666668,10.000000,61.833332,90.0,335.0
3,194.42857,29.833334,35.333332,69.333336,90.0,335.0
4,209.57143,36.833332,5.833334,60.000000,90.0,335.0
...,...,...,...,...,...,...
3386,230.71428,37.833332,-133.285720,230.714280,NaN,NaN
3387,227.14285,39.166668,-136.857150,227.142850,NaN,NaN
3388,224.71428,37.000000,-139.285720,224.714280,NaN,NaN
3389,229.71428,33.666668,-134.285720,229.714280,NaN,NaN


In [6]:
#tile_list = [828, 829, 830, 904, 905, 906, 988, 989, 990] # For 3x3 test area only
#tile_list = list(range(2837,2842)) + list(range(2894,2899)) + list(range(2944,2949)) + list(range(2990,2995)) + list(range(3037,3042)) # North Baffin
tile_list = gpd.read_file(r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\UtilityData\Study Areas\Canada\TileSystems\60km\canadaFishnet_CanVec_60km.shp")
tile_list = tile_list[tile_list['HBL'] == 1].index.tolist()
#tile_list = tile_stats.index.tolist() # for all tiles - Note: only have data for HBL tiles
#tile_list = [2990]

wys = 2023
wye = 2024
wy = str(wys)[2:] + str(wye)[2:]

fmask = 'F:/HLS/Canada/Tiles60km/fmask' # Location of initial Fmask tiles downloaded from STAC
#winterYear_tile = 'C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/HBL/TemporalSnow/Fmask_V1/Products2/winterYear_tile' 
winterYear_tile = 'F:/HLS/Canada/Tiles60km/snowDynamics_fmask_v1/winterYear' # Location to save winterYear tiled products
#winterYear_tile = 'F:/HLS/Canada/Tiles60km/test' # Location to save winterYear tiled products

thresh = 2 # During cleaning, require this many consecutive snow/non-snow observations

In [7]:
tile_stats.iloc[tile_list]

,smin_avg,smax_avg,sstart_avg,send_avg,send_last,sstart_first
365,219.85715,39.333332,-64.166664,124.000000,131.0,284.0
366,213.42857,30.166666,-64.000000,122.500000,132.0,284.0
367,213.57143,32.500000,-63.000000,122.166664,138.0,284.0
368,214.42857,32.333332,-61.833332,122.166664,140.0,283.0
369,214.28572,32.666668,-60.500000,122.000000,141.0,285.0
...,...,...,...,...,...,...
1472,242.00000,50.833332,-69.500000,188.333330,201.0,288.0
1542,222.71428,35.000000,-69.166664,149.333330,172.0,286.0
1543,239.57143,45.833332,-68.166664,182.666670,200.0,288.0
1544,248.57143,58.666668,-52.333332,185.833330,195.0,303.0


## Workflow

In [9]:
%%time
for tile in tile_list:

    filename = 'HLS_Fmask_snowDynamics_winterYear' + wy + '_' + str(tile) + '.nc'

    path = os.path.join(winterYear_tile, str(tile)) # Specific folder for tiled product

    if not os.path.exists(path):
        os.makedirs(path) # Create folder to save NetCDF file in if needed

    if not any(filename in nc for nc in os.listdir(path)): # If NetCDF file has not yet been created

        print('---------------------------')
        print('...Tile ' + str(tile_list.index(tile) + 1) + ' (ID: ' + str(tile) + ')' + ' of ' + str(len(tile_list)) + '...')

        # Load annual fmask cubes
        fmask1 = pputil.uploadNC(os.path.join(fmask, str(wys), 'fmask_' + str(tile) + '_' + str(wys) + '.nc'), 
                                 form = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
        fmask2 = pputil.uploadNC(os.path.join(fmask, str(wye), 'fmask_' + str(tile) + '_' + str(wye) + '.nc'), 
                                 form = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
        print('Loaded annual Fmask cubes (' + str(wys) + ': ' + str(len(fmask1)) +  ' time-steps, ' + str(wye) + ': ' + str(len(fmask2)) + ' time-steps).')

        # Convert to snow cube
        snowCube = snow_util.annualFmask2SnowCube(fmask1, fmask2, yrs = 2, verbose = False)
        print('Converted to 2-year snowCube, including merging same-day & removing 99+% unclear time-steps (' + str(len(snowCube)) + ' time-steps).')
    
        # Get observation frequency
        # clearObsPerDay = snow_util.clearObservationsPerDay_WinterYear(snowCube, windows = ['winterYear', 'snowSeason', 'snowFall', 'snowPeak', 'snowMelt'], 
        #                                                               smin_doy = int(tile_stats.loc[tile, 'smin_avg']),
        #                                                               smax_doy = int(tile_stats.loc[tile, 'smax_avg']), 
        #                                                               sstart_doy = int(tile_stats.loc[tile, 'sstart_avg']),
        #                                                               send_doy = int(tile_stats.loc[tile, 'send_avg']), 
        #                                                               verbose = False)
        # #clearObsPerDay = pputil.loadXR(clearObsPerDay)
        # print('Output clearObsPerDay for temporal windows ' + str(tuple([str.split('_')[0] for str in list(clearObsPerDay.keys())])) + '.')
    
        # Identify implausible snow period
        if np.isnan(tile_stats.loc[tile, 'send_last']) == True:
            implausible_snow = []
            print('No implausible snow period for this tile.')
        else: # Week buffer from latest recorded end date and earliest recorded start date
            implausible_snow = [int(tile_stats.loc[tile, 'send_last']) + 7, int(tile_stats.loc[tile, 'sstart_first']) - 7]
            
            date1 = dt.datetime(wys, 12, 31) + dt.timedelta(implausible_snow[0])
            date1 = str(date1.month) + '/' + str(date1.day)
    
            date2 = dt.datetime(wys, 12, 31) + dt.timedelta(implausible_snow[1])
            date2 = str(date2.month) + '/' + str(date2.day)
            
            print('Implausible snow period identified as ' + date1 + ' (doy ' + str(implausible_snow[0]) + ') to ' + date2 + ' (doy ' + str(implausible_snow[1]) + ').')
    
        # Clean snow cube
        snowCube = snow_util.cleanSnowCube(snowCube,  form = 'binary', temporal = 'gaps', implausible_snow = implausible_snow, thresh = thresh, verbose = False)
        print('Cleaned snowCube, including setting non-snow during implausible snow period & removing snow/non-snow periods below ' + str(thresh) + ' consecutive observations.')
    
        # Convert to daily snow cube
        snowCube = snow_util.dailySnowCube(snowCube, doy = int(tile_stats.loc[tile, 'smin_avg']), verbose = False)
        
        first_day = str(snowCube['time'][0].values)[0:10]
        last_day = str(snowCube['time'][-1].values)[0:10]
        
        print('Converted to winterYear daily snowCube (' + first_day + ' to ' + last_day + ', ' + str(len(snowCube)) + ' time-steps), including linear intepolation to fill gaps.')
    
        # # Load into memory (faster without!)
        # snowCube = pputil.loadXR(snowCube)
        # print('Loaded snowCube into memory.')
    
        # Calculate snow dynamis
        snowDynamics = snow_util.dailySnowCube2SnowDynamics(snowCube, products = ['start', 'end', 'length', 'periods', 'status'], uncertainty = True, verbose = False)
        print('Output winterYear snowDynamics products ' + str(tuple([str[5:] for str in list(snowDynamics.keys())])) + '.')
    
        # Merge clearObsPerDay and snowDynamics into dataset to save
        #snowDynamics = xr.merge([snowDynamics, clearObsPerDay])
        #snowDynamics = snowDynamics.expand_dims(winterYear = xr.Variable('winterYear', [str(wys) + '-' + str(wye)])) # Winter year as time dimension
        #print('Merged winterYear snowDynamics and clearObsPerDay into single dataset (' + str(len(snowDynamics) + ' variables).'))
    
        # Final cleaning and download dataset to NetCDF
        #snowDynamics = snowDynamics.reset_coords(names = ['band'], drop = True) # Remove unnessisary 'band' coordinate
        pputil.downloadNC(snowDynamics, path, filename, form = '')
        print('Final winterYear dataset saved: ' + filename)

        # Memory manageent at end
        del fmask1, fmask2, snowCube, snowDynamics # clearObsPerDay
        client.run(gc.collect) # Collect dask worker garbage
        #client.restart()

---------------------------
...Tile 67 (ID: 825) of 160...
Loaded annual Fmask cubes (2023: 180 time-steps, 2024: 157 time-steps).
Converted to 2-year snowCube, including merging same-day & removing 99+% unclear time-steps (198 time-steps).
Implausible snow period identified as 5/25 (doy 146) to 9/26 (doy 270).
Cleaned snowCube, including setting non-snow during implausible snow period & removing snow/non-snow periods below 2 consecutive observations.
Converted to winterYear daily snowCube (2023-08-13 to 2024-08-12, 366 time-steps), including linear intepolation to fill gaps.
Output winterYear snowDynamics products ('start', 'end', 'length', 'periods', 'status', 'start_u', 'end_u', 'length_u').
Final winterYear dataset saved: HLS_Fmask_snowDynamics_winterYear2324_825.nc
---------------------------
...Tile 68 (ID: 826) of 160...
Loaded annual Fmask cubes (2023: 203 time-steps, 2024: 182 time-steps).
Converted to 2-year snowCube, including merging same-day & removing 99+% unclear time-st

In [ ]:
# The NetCDF saving code can break (rarely) - add retry code to continue after... 

In [7]:
# With load after converting to daily: 692 seconds
# No load: 311 seconds (takes approx 5 miutes per tile per winter year) - If needed can calculate avg. time using date modified for each file

In [8]:
client.shutdown()

## Testing

In [9]:
# fmask1 = pputil.uploadNC(os.path.join(fmask, str(wys), 'fmask_' + str(tile) + '_' + str(wys) + '.nc'), 
#                                  form = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
# fmask1

In [10]:
# fmask2 = pputil.uploadNC(os.path.join(fmask, str(wye), 'fmask_' + str(tile) + '_' + str(wye) + '.nc'), 
#                          form = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
# fmask2

In [11]:
# %%time
# tile = 365

# fmask1 = pputil.uploadNC(os.path.join(fmask, str(wys), 'fmask_' + str(tile) + '_' + str(wys) + '.nc'), 
#                          form = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
# fmask2 = pputil.uploadNC(os.path.join(fmask, str(wye), 'fmask_' + str(tile) + '_' + str(wye) + '.nc'), 
#                          form = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})

# snowCube = snow_util.annualFmask2SnowCube(fmask1, fmask2, yrs = 2, verbose = False)

# implausible_snow = [int(tile_stats.loc[tile, 'send_last']) + 7, int(tile_stats.loc[tile, 'sstart_first']) - 7]

# snowCube = snow_util.cleanSnowCube(snowCube,  form = 'binary', temporal = 'gaps', implausible_snow = implausible_snow, thresh = thresh, verbose = False)

# snowCube = snow_util.dailySnowCube(snowCube, doy = int(tile_stats.loc[tile, 'smin_avg']), verbose = False)
# snowCube

In [12]:
# crs = snowCube.rio.crs # Some xarray functions remove crs 
# snowDynamics = xr.Dataset(coords = dict(x = ('x', snowCube['x'].values), y = ('y', snowCube['y'].values)))
# snowDynamics.rio.write_crs(crs, inplace = True) # Reapply crs
# snowDynamics

In [13]:
# snowCube_b = xr.where(snowCube >= 0.4999, 1, snowCube) # If no uncertainty (e.g., IMS - don't need this)
# snowCube_b = xr.where(snowCube < 0.4999, 0, snowCube_b) # 0.4999 accounts for weird rounding in some cases
# snowCube_b

In [14]:
# from scipy.signal import find_peaks, peak_widths

In [15]:
# test = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] # Start = 6, End = 12

# peaks = find_peaks(test, width = 0)
# #np.float32(peaks[1]['left_bases'][0]) + 2 # +2 because want first 1 (not last 0, +1) & 1 = Jan 1 (not 0, +1)
# # np.float32(peaks[1]['right_bases'][0]) # Want last 1, not next 0, but 1 = Jan 1 so no change

In [16]:
# cumsum = np.cumsum(test)
# cumsum

In [17]:
# np.where(cumsum == 1)[0][0] + 1

In [18]:
# np.argmax(cumsum) + 1

In [19]:
# %%time
# def start_calc(arr):
#     peaks = find_peaks(arr, width = 0)
#     if len(peaks[0]) > 0:
#         start = np.float32(peaks[1]['left_bases'][0]) + 2
#     else: 
#         start = np.nan
#     return start

# # Apply function to array
# results = xr.apply_ufunc(
#     start_calc, 
#     snowCube_b, 
#     input_core_dims = [['time']],
#     vectorize = True,
#     dask = 'parallelized'
# )
# results

In [20]:
# %%time
# results = results.compute()
# results

In [21]:
# snowCube_b['time'][99]

In [22]:
# yr1_days = len(snowCube_b.sel(time = snowCube_b['time'][0].values.astype('datetime64[Y]').astype(str)))
# results1 = results - yr1_days
# results1

In [23]:
#results1.plot() # Looks same as cumsum version

In [24]:
# Snow start
#np.issubdtype(snowCube, np.floating) # True

# snowCube_b = xr.where(snowCube >= 0.4999, 1, snowCube) # If no uncertainty (e.g., IMS - don't need this)
# snowCube_b = xr.where(snowCube < 0.4999, 0, snowCube_b) # 0.4999 accounts for weird rounding in some cases
# b_cumsum = snowCube_b.cumsum(dim = 'time') # Turns to uint64 here
# yr1_days = len(snowCube_b.sel(time = snowCube_b['time'][0].values.astype('datetime64[Y]').astype(str)))
# b_cumsum

In [25]:
# # Start
# snow_start = (xr.where(b_cumsum == 0, 999, b_cumsum).argmin(dim = 'time') + 1) #.astype('float32')
# snow_start = snow_start - yr1_days # Recalculate to days from Dec 31
# snow_start = xr.where(snow_start == 1 - yr1_days, np.nan, snow_start) 
# snow_start.rio.write_crs(crs, inplace = True) # Reapply crs
# snowDynamics['snow_start'] = snow_start.astype('float32') # Add to snowDynamics
# snowDynamics['snow_start']

In [26]:
# %%time
# snowDynamics['snow_start'].compute()

In [27]:
# End
# snow_end = (b_cumsum.argmax(dim = 'time') + 1) # Index starts at 0
# snow_end = snow_end - yr1_days # Recalculate to days from Dec 31
# snow_end = xr.where((snow_end == len(b_cumsum) - yr1_days) | (snow_end == 1 - yr1_days), np.nan, snow_end) 
# snow_end.rio.write_crs(crs, inplace = True) # Reapply crs
# snowDynamics['snow_end'] = snow_end.astype('float32') # Add to snowDynamics  
# snowDynamics['snow_end']

In [28]:
# # Uncertainty
# snowCube_u = xr.where((snowCube < 0.005) | (snowCube > 0.995), 0, snowCube)
# snowCube_u = xr.where((snowCube >= 0.005) & (snowCube <= 0.995) , 1, snowCube_u) 
# snowCube_u.rio.write_crs(crs, inplace = True) # Reapply crs
# snowCube_u

In [29]:
# %%time

# #find_peaks(test[:, 90:110, 990:1010], width = 0)

# def uncertainty(arr): 
#     peaks = find_peaks(arr, width = 0)
#     if len(peaks[0]) > 0: # If their are any peaks (not all 0s)
#         start = peaks[1]['widths'][0] # np.float32()
#         #end = peaks[1]['widths'][-1]
#     else:
#         start = 0
#         #end = 0
#     return start#, end

# # Apply function to array
# results = xr.apply_ufunc(
#     uncertainty, 
#     snowCube_u,
#     input_core_dims = [['time']],
#     vectorize = True,
#     dask = 'parallelized',
#     output_dtypes = [snowCube_u.dtype]
# )
# results

In [30]:
# %%time
# results = results.compute() # 3.5 mins
# results

In [31]:
# from numba import float32, guvectorize#, vectorize#, njit

In [32]:
# import numba as nb

In [33]:
# results.nbytes / 1e9

In [34]:
# %load_ext Cython

# %%cython

In [35]:
# %%time

# #@njit(cache=True) 
# @guvectorize('(float32[:], float32[:])', '(n) -> (n)', nopython = True)
# def uncertainty(arr, out): 
#     peaks = find_peaks(arr, width = 0)
#     if len(peaks[0]) > 0: # If their are any peaks (not all 0s)
#         out[:] = peaks[1]['widths'][0] # np.float32()
#         #end = peaks[1]['widths'][-1]
#     else:
#         out[:] = 0
#         #end = 0

# # Apply function to array
# results = xr.apply_ufunc(
#     uncertainty, 
#     snowCube_u,
#     input_core_dims = [['time']],
#     dask = 'parallelized',
#     output_dtypes = [snowCube_u.dtype]
# )
# results

In [36]:
# %%time
# def start_calc(arr):
#     peaks = find_peaks(arr, width = 0)
#     if len(peaks[0]) > 0:
#         start = np.float32(peaks[1]['left_bases'][0]) + 2
#     else: 
#         start = np.nan
#     return start

# # Apply function to array
# results = xr.apply_ufunc(
#     start_calc, 
#     snowCube_b, 
#     input_core_dims = [['time']],
#     vectorize = True,
#     dask = 'parallelized'
# )
# results

In [37]:
# %%time
# snow_start_inmem = snow_start.compute() # Needed to avoid double load if quantiles are required... 
# snow_end_inmem = snow_end.compute() # Needed to avoid double load if quantiles are required...

In [38]:
# snow_start_inmem.plot()

In [39]:
# snow_end_inmem.plot()

In [40]:
# start_last = int(snow_start_inmem.max()) # Latest snow start date in area
# start_last

In [41]:
# end_first = int(snow_end_inmem.min()) 
# end_first

In [42]:
# endstart_diff = end_first - start_last 
# endstart_diff

In [43]:
# %%time
# start_last = int(snow_start_inmem.quantile(0.99)) # Latest* snow start date in area
# start_last

In [44]:
# %%time
# end_first = int(snow_end_inmem.quantile(0.01)) # Earliest* snow end date in area
# end_first

In [45]:
# endstart_diff = end_first - start_last # Difference between
# endstart_diff

In [46]:
# start_last = int(snow_start_inmem.quantile(0.98)) 
# start_last

In [47]:
# end_first = int(snow_end_inmem.quantile(0.02))
# end_first

In [48]:
# endstart_diff = end_first - start_last # Difference between
# endstart_diff

In [49]:
# tile_stats.iloc[tile]

In [50]:
# start_last = dt.datetime(int(snowCube.time.dt.year[-1] - 1), 12, 31) + dt.timedelta(start_last)
# end_first = dt.datetime(int(snowCube.time.dt.year[-1] - 1), 12, 31) + dt.timedelta(end_first)
# split_date = start_last + (end_first - start_last) / 2 # Split date
# split_date

In [51]:
# split_date_1 = split_date + dt.timedelta(1) # Day after split date for filtering
# split_date =  f'{split_date.year}-{split_date.month:02d}-{split_date.day:02d}' # String for filtering
# split_date

In [52]:
# snow_start_inmem.plot.hist()

In [53]:
# snow_end_inmem.plot.hist()

In [54]:
# snowCube_u[:, 100,100].plot()

In [55]:
# test = snowCube_u.compute()
# test

In [56]:
#test[:, 100,100].sum() / 2
#test[:, 100,100].sel(time = slice(None, split_date)).sum(dim = 'time') / 2

In [57]:
# split_date_1 = f'{split_date_1.year}-{split_date_1.month:02d}-{split_date_1.day:02d}'
# test[:, 100,100].sel(time = slice(split_date_1, None)).sum(dim = 'time') / 2 

In [58]:
# test[:, 100,1000].plot()

In [59]:
# %%time
# peaks = find_peaks(test[:, 100,1000], width = 0)
# peaks

In [60]:
#peaks[1]['widths'][0]

In [61]:
#peaks[1]['widths'][-1]

In [62]:
# peaks, _ = find_peaks(test[:, 100,1000])
# peaks

In [63]:
# widths = peak_widths(test[:, 100,1000], peaks)
# widths[0][0] / 2

In [64]:
# widths[0][-1] / 2

In [65]:
#snowCube_b[:, 100, 1000].plot()

In [66]:
# peaks = find_peaks(snowCube_b[:, 100, 1000], width = 0)
# peaks

In [67]:
#uncertainty(test[:, 100, 1000])

In [68]:
#peaks[1]['widths'][0]

In [69]:
#peaks[1]['widths'][-1]

In [70]:
#snowCube_b

In [71]:
#test

In [72]:
#uncertainty(test[0:50, 100, 100]) # 0:50

In [73]:
#len(peaks[0])

In [74]:
#test[:, 100, 1000].values

In [75]:
#test.isel(x = 100, y = 100)

In [76]:
#test#.isnull().sum()

In [77]:
#results.plot()

In [78]:
# import numpy as np
# import xarray as xr
# from scipy.signal import find_peaks

# # Generate waveform
# x = (np.sin(2*np.pi*(2**np.linspace(2,10,1000))*np.arange(1000)/48000)
#      + np.random.normal(0, 1, 1000) * 0.15)
# #x

# # Find peaks non-xarray way
# peaks, _ = find_peaks(x, prominence=1)
# print(peaks)

# # Cast waveform to xr.DataArray
# x = xr.DataArray(x, dims='time')
# #x

# # Duplicate data along a new dimension
# rep = xr.DataArray(range(11), dims='repeat')
# x = (x.broadcast_like(rep).assign_coords(repeat=rep))
# x

# # def process_peaks(arr):
# #     # Apply find_peaks
# #     peaks, _ = find_peaks(arr, prominence=1)
# #     return peaks

# # # Apply function to array
# # results = xr.apply_ufunc(
# #     process_peaks, x,
# #     input_core_dims=[['time']],
# #     output_core_dims=[['peaks']],
# #     vectorize=True
# # )

# # # Should show repeats of peak results
# # print(results)

In [79]:
#results

In [80]:
# u_cumsum = test.cumsum(dim = 'time')
# u_cumsum

In [81]:
#u_cumsum[:, 100,100].plot()

In [82]:
# cumsum_r = u_cumsum - u_cumsum.where(test == 0).ffill(dim = 'time').fillna(0)
# cumsum_r

In [83]:
#cumsum_r[:, 100,100].plot()

In [84]:
# Error on snow_start.quantile(0.99) related to dask chunking for one specific tile and winterYear...

# tile = 989

# # Run everything up to dailySnowCube2SnowDynamis:
# # Load annual fask cubes
# fmask1 = pputil.uploadNC(os.path.join(fmask, str(wys), 'fmask_' + str(tile) + '_' + str(wys) + '.nc'), 
#                          type = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
# fmask2 = pputil.uploadNC(os.path.join(fmask, str(wye), 'fmask_' + str(tile) + '_' + str(wye) + '.nc'), 
#                          type = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
# print('Loaded annual Fmask cubes (' + str(wys) + ': ' + str(len(fmask1)) +  ' time-steps, ' + str(wye) + ': ' + str(len(fmask2)) + ' time-steps).')

# # Convert to snow cube
# snowCube = snow_util.annualFmask2SnowCube(fmask1, fmask2, yrs = 2, verbose = False)
# print('Converted to 2-year snowCube, including merging same-day & removing 99+% unclear time-steps (' + str(len(snowCube)) + ' time-steps).')

# # Get observation frequency
# clearObsPerDay = snow_util.clearObservationsPerDay_WinterYear(snowCube, windows = ['winterYear', 'snowSeason', 'snowFall', 'snowPeak', 'snowMelt'], 
#                                                               smin_doy = int(tile_stats.loc[tile, 'smin_avg']),
#                                                               smax_doy = int(tile_stats.loc[tile, 'smax_avg']), 
#                                                               sstart_doy = int(tile_stats.loc[tile, 'sstart_avg']),
#                                                               send_doy = int(tile_stats.loc[tile, 'send_avg']), 
#                                                               verbose = False)
# #clearObsPerDay = pputil.loadXR(clearObsPerDay)
# print('Output clearObsPerDay for temporal windows ' + str(tuple([str.split('_')[0] for str in list(clearObsPerDay.keys())])) + '.')

# # Identify implausible snow period
# if np.isnan(tile_stats.loc[tile, 'send_last']) == True:
#     implausible_snow = []
#     print('No implausible snow period for this tile.')
# else: # Week buffer from latest recorded end date and earliest recorded start date
#     implausible_snow = [int(tile_stats.loc[tile, 'send_last']) + 7, int(tile_stats.loc[tile, 'sstart_first']) - 7]
    
#     date1 = dt.datetime(wys, 12, 31) + dt.timedelta(implausible_snow[0])
#     date1 = str(date1.month) + '/' + str(date1.day)

#     date2 = dt.datetime(wys, 12, 31) + dt.timedelta(implausible_snow[1])
#     date2 = str(date2.month) + '/' + str(date2.day)
    
#     print('Implausible snow period identified as ' + date1 + ' (doy ' + str(implausible_snow[0]) + ') to ' + date2 + ' (doy ' + str(implausible_snow[1]) + ').')

# # Clean snow cube
# snowCube = snow_util.cleanSnowCube(snowCube,  type = 'binary', temporal = 'gaps', implausible_snow = implausible_snow, thresh = thresh, verbose = False)
# print('Cleaned snowCube, including setting non-snow during implausible snow period & removing snow/non-snow periods below ' + str(thresh) + ' consecutive observations.')

# # Convert to daily snow cube
# snowCube = snow_util.dailySnowCube(snowCube, doy = int(tile_stats.loc[tile, 'smin_avg']), verbose = False)

# first_day = str(snowCube['time'][0].values)[0:10]
# last_day = str(snowCube['time'][-1].values)[0:10]

# print('Converted to winterYear daily snowCube (' + first_day + ' to ' + last_day + ', ' + str(len(snowCube)) + ' time-steps), including linear intepolation to fill gaps.')

In [85]:
# snowCube

In [86]:
# # In dailySnowCube2SnowDynamics...
# crs = snowCube.rio.crs # Some xarray functions remove crs 

# snowCube_b = xr.where(snowCube >= 0.4999, 1, snowCube) # If no uncertainty (e.g., IMS - don't need this)
# snowCube_b = xr.where(snowCube < 0.4999, 0, snowCube_b) # 0.4999 accounts for weird rounding in some cases

# b_cumsum = snowCube_b.cumsum(dim = 'time')
# yr1_days = len(snowCube_b.sel(time = snowCube_b['time'][0].values.astype('datetime64[Y]').astype(str)))

# snow_start = (xr.where(b_cumsum == 0, 999, b_cumsum).argmin(dim = 'time') + 1)
# snow_start = snow_start - yr1_days # Recalculate to days from Dec 31
# snow_start = xr.where(snow_start == 1 - yr1_days, np.nan, snow_start) 
# snow_start.rio.write_crs(crs, inplace = True) # Reapply crs

# snow_start

In [87]:
# snow_end = (b_cumsum.argmax(dim = 'time') + 1) # Index starts at 0
# snow_end = snow_end - yr1_days # Recalculate to days from Dec 31
# snow_end = xr.where((snow_end == len(b_cumsum) - yr1_days) | (snow_end == 1 - yr1_days), np.nan, snow_end) 
# snow_end.rio.write_crs(crs, inplace = True) # Reapply crs

# snow_end

In [88]:
# snow_start_inmem = snow_start.compute()
# snow_end_inmem = snow_end.compute()
# start_last = int(snow_start_inmem.max()) # Latest snow start date in area
# end_first = int(snow_end_inmem.min()) # Earliest snow end date in area
# endstart_diff = end_first - start_last # Difference between
# endstart_diff # Requires load into memory, twice! Took 154 seconds...

In [89]:
# #start_last = int(inmem.chunk({'x': -1, 'y': -1}).quantile(0.99))
# #end_first = int(snow_end.chunk({'x': -1, 'y': -1}).quantile(0.01))
# start_last = int(snow_start_inmem.quantile(0.99))
# end_first = int(snow_end_inmem.quantile(0.01))
# endstart_diff = end_first - start_last
# endstart_diff # Also requires loading into memory, twice... Took 152 seconds. Instant if leoading start/end into memory above

In [90]:
# # Convert latest start and earliest end to dates, and find split date between
# start_last = dt.datetime(int(snowCube.time.dt.year[-1] - 1), 12, 31) + dt.timedelta(start_last)
# end_first = dt.datetime(int(snowCube.time.dt.year[-1] - 1), 12, 31) + dt.timedelta(end_first)
# split_date = start_last + (end_first - start_last) / 2 # Split date
# split_date_1 = split_date + dt.timedelta(1) # Day after split date for filtering
# split_date =  f'{split_date.year}-{split_date.month:02d}-{split_date.day:02d}' # String for filtering
# split_date

In [91]:
# tile_stats.loc[tile]

In [92]:
# (end_first - start_last) / 2

In [93]:
# snow_start.max() + 1

In [94]:
# %%time
# #start_last = snow_start.chunk({'x': -1, 'y': -1}).quantile(0.99) # Latest* snow start date in area
# #end_first = snow_end.chunk({'x': -1, 'y': -1}).quantile(0.01) # Earliest* snow end date in 
# start_last = snow_start.max()
# end_first = snow_end.min()

In [95]:
# %%time
# endstart_diff = end_first - start_last 
# endstart_diff

In [96]:
# %%time
# if endstart_diff >= 0:
#     print('hi')

In [97]:
# %%time
# start_last = dt.datetime(int(snowCube.time.dt.year[-1] - 1), 12, 31) + dt.timedelta(start_last)
# start_last

In [98]:
# %%time
# tile = 1471
# fmask1 = pputil.uploadNC(os.path.join(fmask, str(wys), 'fmask_' + str(tile) + '_' + str(wys) + '.nc'), 
#                          type = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
# fmask2 = pputil.uploadNC(os.path.join(fmask, str(wye), 'fmask_' + str(tile) + '_' + str(wye) + '.nc'), 
#                          type = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
# #snowCube = snow_util.annualFmask2SnowCube(fmask1, fmask2, yrs = 2, verbose = False) # Empty cube
# cube_yr1 = fmask1.groupby('time').min(engine = 'flox', skipna = True)
# cube_yr2 = fmask2.groupby('time').min(engine = 'flox', skipna = True)
# snowCube = xr.concat((cube_yr1, cube_yr2), dim = 'time').sortby('time').squeeze() 

# # snowCube = pputil.timestepClean(snowCube, snowCube.isin([1,2,3,4,5,6]), timestepClean = 'unclear', thresh = 99, verbose = True)
# # snowCube # Old timestepClean: 14 seconds, 120 time-steps

# # valid = xr.where(snowCube[0] != 0, 1, 0).sum(dim = ['x', 'y'])#[:,0] #.values[:,0]
# # clear = xr.where(snowCube.isin([1,2,3,4,5,6]), 1, 0).sum(dim = ['x', 'y'])#[:,0] #.values[:,0]
# # snowCube = snowCube.sel(time = (clear / valid * 100) > 100 - 99)
# # snowCube 
# # New timestepClean, with values: 20 seconds, 140 time-steps
# # New timestepClean, without values: 15 seconds, 140 time-steps
# # New timestepClean, without values, static valid: 13 seconds, 140 time-steps, only works in scenarios where valid pixels are static across time

# snowCube = pputil.timestepClean(snowCube, snowCube.isin([1,2,3,4,5,6]), timestepClean = 'unclear', valid_status = 'static', invalid = 0, thresh = 99, verbose = True)
# snowCube # Static = 13 seconds, # all = 14 seconds

In [99]:
# %%time
# #all = snowCube.shape[-2] * snowCube.shape[-1] * 8.72389282514 / 100 # 349304.75595753384 (estimated), instant
# #all = xr.where(snowCube[0] != 0, 1, 0).sum().values # 350,029 (true count), ~3 seconds
# valid = xr.where(snowCube != 0, 1, 0).sum(dim = ['x', 'y']).values[:,0] # 350,029 (true count), ~10 seconds
# valid 

In [100]:
# arr = np.full(snowCube.shape[0], True) # bool array with len = time-steps
# arr

In [101]:
# %%time
# #clear = np.sum(snowCube.isin([1,2,3,4,5,6]), axis = tuple(range(1, snowCube.isin([1,2,3,4,5,6]).ndim))).values
# clear = xr.where(snowCube.isin([1,2,3,4,5,6]), 1, 0).sum(dim = ['x', 'y']).values[:,0]
# clear 

In [102]:
# %%time
# for timestep in range(snowCube.shape[0]): # For each time-step...
#     if 100 - ((clear[timestep] / all) * 100) > 99: # Get % unclear, compare with threshold
#         arr[timestep] = False
# snowCube = snowCube.sel(time = arr) # Keep only time-steps below thresh
# snowCube # 140 time-steps

In [103]:
# tile = tile_list[0]
# tile

In [104]:
# # Load annual fask cubes
# fmask1 = pputil.uploadNC(os.path.join(fmask, str(wys), 'fmask_' + str(tile) + '_' + str(wys) + '.nc'), 
#                          type = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
# fmask2 = pputil.uploadNC(os.path.join(fmask, str(wye), 'fmask_' + str(tile) + '_' + str(wye) + '.nc'), 
#                          type = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
# fmask2

In [105]:
# # Convert to snow cube
# snowCube = snow_util.annualFmask2SnowCube(fmask1, fmask2, yrs = 2, verbose = False)
# snowCube

In [106]:
# implausible_snow = [int(tile_stats.loc[tile, 'send_last']) + 7, int(tile_stats.loc[tile, 'sstart_first']) - 7]
# implausible_snow

In [107]:
# snowCube = snow_util.cleanSnowCube(snowCube,  type = 'binary', temporal = 'gaps', implausible_snow = implausible_snow, thresh = thresh, verbose = False)
# snowCube

In [108]:
# snowCube = snow_util.dailySnowCube(snowCube, doy = int(tile_stats.loc[tile, 'smin_avg']), verbose = False)
# snowCube

In [109]:
# snowDynamics = snow_util.dailySnowCube2SnowDynamics(snowCube, products = ['start', 'end', 'length', 'periods', 'status'], uncertainty = True, verbose = False)
# snowDynamics

In [110]:
# # Final cleaning and save dataset to TIFs
# snowDynamics = snowDynamics.reset_coords(names = ['band'], drop = True) # Remove unnessisary 'band' coordinate

# variables = list(snowDynamics.keys()) # Name of all data variables as list 

# # Load into memory
# snowDynamics = pputil.loadXR(snowDynamics)
# snowDynamics

In [111]:
# test = snowDynamics['snow_end']

In [112]:
# test.rio.write_nodata(-999, encoded = True, inplace = True) # Ensured NaN NoData values are saved
# test

In [113]:
# import numpy as np
# np.unique(test, return_counts = True)

In [114]:
# for variable in variables:

#     da = snowDynamics[variable].astype('float32') # Single snow dynamic stat from dataset

#     da.rio.write_nodata(-999, encoded = True, inplace = True) # Ensured NaN NoData values are saved
#     da.rio.to_raster(os.path.join(path, str(wy), 'HLS_Fmask_' + variable + '_' + output_ID + '.tif'))        

#     print('Product ' + str(variables.index(variable) + 1) + ' of ' + str(len(variables)) + ' saved: HLS_Fmask_' + variable + '_' + output_ID + '.tif')

In [115]:

        # snowDynamics = snowDynamics.expand_dims(winterYear = xr.Variable('winterYear', [str(wys) + '-' + str(wye)])) # Winter year as time dimension
        # snowDynamics = snowDynamics.reset_coords(names = ['band'], drop = True) # Remove unnessisary 'band' coordinate
        # pputil.downloadNC(snowDynamics, path, filename, type = '')
        # print('Final winterYear dataset saved: ' + filename)

        # # Memory manageent at end
        # del fmask1, fmask2, snowCube, snowDynamics # clearObsPerDay
        # client.run(gc.collect) # Collect dask worker garbage
        # #client.restart()